In [1]:
import pandas as pd
from loaddata import defineTestSet,defineResponse,defineFeatures,defineTrainingSets,defineSplits
import numpy as np
import pickle

from tools.model_roc import get_model_roc

from sklearn.model_selection import RandomizedSearchCV

from config import rand_var,ml_dir,ml_model_filename,ensemble_dir

import os
from datetime import datetime

In [11]:


for i,(tr,ts) in enumerate(splits):
    
    xtrain_tr = Xtrain.iloc[tr,:]
    ytrain_tr = ytrain_pCR.iloc[tr]
    
    
    ml_dict["Logistic Regression"].fit(xtrain_tr,ytrain_tr) 
    ml_dict["Random Forest Classifier"].fit(xtrain_tr,ytrain_tr) 
    ensemble_model.fit(xtrain_tr,ytrain_tr) 
    
    y_pred_logres = ml_dict["Logistic Regression"].predict(Xtrain.iloc[ts,:])
    y_pred_gb = ml_dict["Random Forest Classifier"].predict(Xtrain.iloc[ts,:])
    #y_pred_ensemble = ensemble_model.predict(Xtrain.iloc[ts,:])
 
    #fp_rate, tp_rate, thresholds = roc_curve(ytrain_pCR.iloc[ts], y_pred)
    
    #roc_auc = auc(fp_rate, tp_rate)
    #print(roc_auc)
    
    break


